# Welcome to the Spark RAPIDS NDS Demo Lab
This notebook will guide you through understanding how to run GPU [NDS queries](https://github.com/NVIDIA/spark-rapids-benchmarks/tree/dev/nds) using Spark RAPIDS.  


## View GPU configs
To view the configurations used for the Spark RAPIDS job using GPU resources, you can see the various Spark settings that will be used in the job (and later tuned).  Many of the settings are what would be default, but are shown here for completeness.

Note that settings with the prefix "spark.rapids" are specific to using the Spark RAPIDS plugin.

In [ ]:
%%script env bash
cd /tmp
cat <<EOT > power_run_gpu.template

export SPARK_HOME=${SPARK_HOME:-/usr/lib/spark}
export SPARK_RAPIDS_PLUGIN_JAR=../../rapids-4-spark_2.12-23.12.1-cuda11.jar
export NDS_LISTENER_JAR=${NDS_LISTENER_JAR:-./jvm_listener/target/nds-benchmark-listener-1.0-SNAPSHOT.jar}
export PYTHONPATH=$SPARK_HOME/python:`echo $SPARK_HOME/python/lib/py4j-*.zip`

export SPARK_CONF=("--conf" "spark.plugins=com.nvidia.spark.SQLPlugin"
                   "--conf" "spark.sql.files.maxPartitionBytes=128mb"
                   "--conf" "spark.rapids.sql.concurrentGpuTasks=1"
                   "--conf" "spark.eventLog.enabled=true"
                   "--conf" "spark.eventLog.dir=/tmp/spark-events"
                   "--conf" "spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem"
                   "--conf" "spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"
                   "--conf" "spark.jars.packages=org.apache.hadoop:hadoop-aws:3.2.2,com.amazonaws:aws-java-sdk-bundle:1.12.180"
                   "--files" "\$SPARK_HOME/examples/src/main/scripts/getGpusResources.sh"
                   "--jars" "\$SPARK_RAPIDS_PLUGIN_JAR,\$NDS_LISTENER_JAR")
EOT

## View SQL query
You can view the SQL query syntax that will be executed as a Spark RAPIDS application using GPU resources.  Note that the queries are identical whether running on CPU or using Spark RAPIDS on GPU.

In [ ]:
query="4"

In [ ]:
%%script env query="$query" bash
cat ./query_files/query$query.sql

## Run GPU application
The SQL query will be executed as a Spark RAPIDS job on your instance.  It will read data from S3 at a scale factor of 100GB and compute the query using GPU resources as applicable.  Once the query completes, it will output the entire duration for the run, including the table setup time as well as the individual query execution time.

In [ ]:
%%script env query="$query" bash
cd ./spark-rapids-benchmarks/nds
./spark-submit-template /tmp/power_run_gpu.template \
    nds_power.py \
    s3a://dli-public-datasets-us-west-2/production/x-ds-04-v1/parquet_sf100 \
    ../../query_files/query$query.sql \
    gpu-time-tuning-$query.csv

In [ ]:
import pandas as pd

gpu_original_times = pd.read_csv("/dli/task/spark-rapids-benchmarks/nds/gpu-time-" + query + ".csv")
gpu_tuning_times = pd.read_csv("/dli/task/spark-rapids-benchmarks/nds/gpu-time-tuning-" + query + ".csv")

original_total_time = gpu_original_times[gpu_original_times['query'] == "Total Time"]['time/milliseconds'].values[0]
tuning_total_time = gpu_tuning_times[gpu_tuning_times['query'] == "Total Time"]['time/milliseconds'].values[0]
tuning_total_speedup = original_total_time / tuning_total_time

print("================================")
print("Query " + query + " Total Speedup Results")
print("================================")
print("Original Total Time = " + str(original_total_time))
print("Tuning Total Time = " + str(tuning_total_time))
print("Acceleration = " + str(round((tuning_total_speedup),2)))

query_name = "query" + query
original_query_time = gpu_original_times[gpu_original_times['query'] == query_name]['time/milliseconds'].values[0]
tuning_query_time = gpu_tuning_times[gpu_tuning_times['query'] == query_name]['time/milliseconds'].values[0]
tuning_query_speedup = original_query_time / tuning_query_time

print("")
print("================================")
print("Query " + query + " SQL Speedup Results")
print("================================")
print("Original Query Time = " + str(original_query_time))
print("Tuning Query Time = " + str(tuning_query_time))
print("Acceleration = " + str(round((tuning_query_speedup),2)))

The actual acceleration will depend on the specific hardware, but a 2x+ acceleration can be expected based on the previous analysis. 